In [ ]:
# As my dataset is in google drive and i am using colab for gpu utilization, i am mounting the drive to colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Installing the requirements.
!pip install transformers -q
!pip install sentencepiece -q

     |████████████████████████████████| 3.8 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 52.3 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
     |████████████████████████████████| 6.5 MB 43.4 MB/s 
     |████████████████████████████████| 1.2 MB 4.3 MB/s 


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "flexudy/t5-small-wav2vec2-grammar-fixer" # model i have used.

tokenizer = T5Tokenizer.from_pretrained(model_name) #loading the T5Tokenizer from pretrained model

model = T5ForConditionalGeneration.from_pretrained(model_name) # loading the T5ForConditionalGeneration from pretrained model



Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/Datasets/review_data.csv") #reading the dataset

In [ ]:
data.insert(2,"Corrected_scent","") # adding Corrected_scent column to the dataframe at position 3, i.e, after star column

In [ ]:
data_new = data.head(3000) #selecting a sample of data i.e., 10% of the total data for processing as dataset is large[30k]

In [ ]:
data_new

,text,star,Corrected_scent,app_id,reviewDate
0,Anathi Khanyile,5,,com.fingersoft.hillclimb,18/03/21
1,Tony bahut funny hai Hill climbing racing my f...,5,,com.fingersoft.hillclimb,18/03/21
2,Teturwu,1,,com.fingersoft.hillclimb,18/03/21
3,Hoooooooooooyaaaaaaaaa what a game hoooooooooo...,5,,com.fingersoft.hillclimb,18/03/21
4,This game is nice,5,,com.fingersoft.hillclimb,18/03/21
...,...,...,...,...,...
2995,I like the game,5,,com.ea.game.simcitymobile_row,18/03/21
2996,I love it its my favrot game,5,,com.ea.game.simcitymobile_row,18/03/21
2997,Very hard and lots and lots to think about,5,,com.ea.game.simcitymobile_row,18/03/21
2998,I can't tell anything for this game,5,,com.ea.game.simcitymobile_row,18/03/21


In [ ]:
count = 0
for i in range(len(data.text)):
  sent = str(data_new['text'][i]) # iterating and selecting each text data

  input_text = "fix: { " + sent + " } </s>"

  input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=256, truncation=True, add_special_tokens=True) #encoding the input text

  #setting parameters for the model
  outputs = model.generate(
      input_ids=input_ids,
      max_length=256,
      num_beams=4,
      repetition_penalty=1.0,
      length_penalty=1.0,
      early_stopping=True
    )

  sentence = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True) # decoding the corrected scentence

  #print(f"{sentence}")
  data["Corrected_scent"][i] = sentence # saving the corrected scntence into the dataframe 
  count+=1
  print(count)

2941
2942
2943
2944
2945
2946
2947
2948
2949
2950
2951
2952
2953
2954
2955
2956
2957
2958
2959
2960
2961
2962
2963
2964
2965
2966
2967
2968
2969
2970
2971
2972
2973
2974
2975
2976
2977
2978
2979
2980
2981
2982
2983
2984
2985
2986
2987
2988
2989
2990
2991
2992
2993
2994
2995
2996
2997
2998
2999
3000


KeyError: ignored

In [ ]:
data_corrected = data.head(3000)

In [ ]:
data_corrected.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             3000 non-null   object
 1   star             3000 non-null   int64 
 2   Corrected_scent  3000 non-null   object
 3   app_id           3000 non-null   object
 4   reviewDate       3000 non-null   object
dtypes: int64(1), object(4)
memory usage: 117.3+ KB


In [ ]:
data_corrected['Corrected_scent'].isnull().sum() # checking for any null value present or not

0

In [18]:
data_corrected.to_csv('output_file.csv')# saving the file

In [19]:
import pandas as pd
data_corrected = pd.read_csv('/content/output_file.csv')

In [20]:
data_corrected

,Unnamed: 0,text,star,Corrected_scent,app_id,reviewDate
0,0,Anathi Khanyile,5,Anathi Khanyile,com.fingersoft.hillclimb,18/03/21
1,1,Tony bahut funny hai Hill climbing racing my f...,5,"Tony Bahut Funny Hai Hill climbing, My favouri...",com.fingersoft.hillclimb,18/03/21
2,2,Teturwu,1,Teturwu -,com.fingersoft.hillclimb,18/03/21
3,3,Hoooooooooooyaaaaaaaaa what a game hoooooooooo...,5,Hooooooooooooooooooooooooooooooooooooooooooooo...,com.fingersoft.hillclimb,18/03/21
4,4,This game is nice,5,This game is nice.,com.fingersoft.hillclimb,18/03/21
...,...,...,...,...,...,...
2995,2995,I like the game,5,I like the game.,com.ea.game.simcitymobile_row,18/03/21
2996,2996,I love it its my favrot game,5,"I love it, it's my favrot game.",com.ea.game.simcitymobile_row,18/03/21
2997,2997,Very hard and lots and lots to think about,5,Very hard and lots and lots to think about.,com.ea.game.simcitymobile_row,18/03/21
2998,2998,I can't tell anything for this game,5,I can't tell anything for this game.,com.ea.game.simcitymobile_row,18/03/21
